# Normative modelling as site effect correction tool

This tutorial will walk you through the application of normative modelling as site effect correction tool.

You will learn:
- How to fit a normative model to data from different sites
- choices to make about the model
- how to transfer data from one site into another
- how to make predictions for unseen sites and the rationale behind it.

# Quick recap: normative models using the pcn toolkit
A normative model requires three input variables:
- covariates (the X axis, ususally age)
- response variables (usually one or several brain feature measurements)
- a batch effect file. Batch effects can be anything that interferes with the covrariate (sex, site)

While the blr model reuiqres input of the batch effects via a design matrix, the implementation of the hbr only reuires a file.

In [ ]:
# Import relevant libraries
import pcntoolkit
import os
import pandas as pd
import pcntoolkit as ptk
import numpy as np
import pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix

In [ ]:
!pwd

In [ ]:
# create working dir
root_dir = os.getcwd()

In [ ]:
print(root_dir)

In [ ]:
processing_folder = "Site_effect_tutorial/"
os.chdir(processing_folder)

In [ ]:
if not os.path.isdir(processing_folder):
    os.makedirs(processing_folder)

In [ ]:
#os.chdir(tutdir)
os.chdir(processing_folder)

In [ ]:
processing_dir = os.getcwd()
print(f"The processing directory is: {processing_dir}")

# 01. Data pre-processing.

### Load the data

For this tutorial, we use publicly availabe data that has been pooled from various data sets.

In [ ]:
# Load the data
!wget -nc https://raw.githubusercontent.com/saigerutherford/CPC_ML_tutorial/master/data/fcon1000_tr.csv
!wget -nc https://raw.githubusercontent.com/saigerutherford/CPC_ML_tutorial/master/data/fcon1000_te.csv

In [ ]:
# Load the test data into the notebook
test_data = os.path.join(root_dir, 'fcon1000_te.csv')
df_te = pd.read_csv(test_data, index_col=0)

# remove some bad subjects
# df_te, bad_sub = remove_bad_subjects(df_te, df_te)

# extract a list of unique site ids from the test set
site_ids_te =  sorted(set(df_te['site'].to_list()))

In [ ]:
# Load the training data into the notebook
train_data = os.path.join(root_dir, 'fcon1000_tr.csv')

df_tr = pd.read_csv(train_data, index_col=0)

# remove some bad subjects
#df_ad, bad_sub = remove_bad_subjects(df_ad, df_ad)

# extract a list of unique site ids from the test set
site_ids_ad =  sorted(set(df_tr['site'].to_list()))

if not all(elem in site_ids_ad for elem in site_ids_te):
    print('Warning: some of the testing sites are not in the adaptation data')

### Make a quick display of the age distibution per site.

In [ ]:
axes = df_tr.hist(['age'], by='site', layout=(5,5),
                  legend=True, yrot=90,sharex=True,sharey=True, 
                  figsize=(6,6))
for ax in axes.flatten():
    ax.set_xlabel('age')
    ax.set_ylabel('distribution')
    ax.set_ylim(bottom=1,top=25)

### Additionally, print the number of subjects per site.

In [ ]:
sites = df_tr['site'].unique()

print('sample size check')
for i,s in enumerate(sites):
    idx = df_tr['site'] == s
    idxte = df_te['site'] == s
    print(i,s, sum(idx), sum(idxte))


In [ ]:
df_tr['sitenum'] = 0
for i,s in enumerate(sites):
    idx = df_tr['site'] == s
    df_tr['sitenum'].loc[idx] = i

    print('site',s, sum(idx))

In [ ]:
df_te['sitenum'] = 0
for i,s in enumerate(sites):
    idx = df_te['site'] == s
    df_te['sitenum'].loc[idx] = i

    print('site',s, sum(idx))

### Set aside hold-out transfer sites.
For our site transfer later we want to exclude some sites from the training and test set for later.

In [ ]:
# Save some sites for later, these are going to be my new sites
new_sites_tr = df_tr[(df_tr['site']== 'SaintLouis')| (df_tr['site']=='COBRE')]
new_siets_te = df_te[(df_te['site']== 'SaintLouis')| (df_te['site']=='COBRE')]

In [ ]:
# Remove those sites from the training and test sets
df_tr = df_tr[df_tr.site != 'SaintLouis']
df_tr = df_tr[df_tr.site != 'COBRE']
df_te = df_te[df_te.site != 'SaintLouis']
df_te = df_te[df_te.site != 'COBRE']

In [ ]:
# Select the IDPs (columns) of interest from the data frame
idps = ['lh_G&S_frontomargin_thickness','rh_G&S_frontomargin_thickness']

In [ ]:
# Prepare the data frames for training and testing

X_train = (df_tr['age']/100).to_numpy(dtype=float)
Y_train = df_tr[idps].to_numpy(dtype=float)
batch_effects_train = df_tr[['sitenum', 'sex']].to_numpy(dtype=int)

#test_data[['site_id','sex']].to_numpy(dtype=float)

# save data
with open('X_train.pkl', 'wb') as file:
    pickle.dump(pd.DataFrame(X_train), file)
with open('Y_train.pkl', 'wb') as file:
    pickle.dump(pd.DataFrame(Y_train), file)
with open('trbefile.pkl', 'wb') as file:
    pickle.dump(pd.DataFrame(batch_effects_train), file)


X_test = (df_te['age']/100).to_numpy(dtype=float)
Y_test = df_te[idps].to_numpy(dtype=float)
batch_effects_test = df_te[['sitenum', 'sex']].to_numpy(dtype=int)

#save data
with open('X_test.pkl', 'wb') as file:
    pickle.dump(pd.DataFrame(X_test), file)
with open('Y_test.pkl', 'wb') as file:
    pickle.dump(pd.DataFrame(Y_test), file)
with open('tsbefile.pkl', 'wb') as file:
    pickle.dump(pd.DataFrame(batch_effects_test), file)

# a simple function to quickly load pickle files
def ldpkl(filename: str):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [ ]:
# a simple function to quickly load pickle files
def ldpkl(filename: str):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [ ]:
# the model needs the paths to the data. create them here

respfile = os.path.join(processing_dir, 'Y_train.pkl')       # measurements  (eg cortical thickness) of the training samples (columns: the various features/ROIs, rows: observations or subjects)
covfile = os.path.join(processing_dir, 'X_train.pkl')        # covariates (eg age) the training samples (columns: covariates, rows: observations or subjects)

testrespfile_path = os.path.join(processing_dir, 'Y_test.pkl')       # measurements  for the testing samples
testcovfile_path = os.path.join(processing_dir, 'X_test.pkl')        # covariate file for the testing samples

trbefile = os.path.join(processing_dir, 'trbefile.pkl')      # training batch effects file (eg scanner_id, gender)  (columns: the various batch effects, rows: observations or subjects)
tsbefile = os.path.join(processing_dir, 'tsbefile.pkl')      # testing batch effects file

output_path = os.path.join(processing_dir, 'Models/')    #  output path, where the models will be written
log_dir = os.path.join(processing_dir, 'log/')           # log path
if not os.path.isdir(output_path):
    os.mkdir(output_path)
if not os.path.isdir(log_dir):
    os.mkdir(log_dir)

outputsuffix = 'estimate'      # a string to name the output files, of use only to you, so adapt it for your needs.

# 02. Normative modelling
## Case 1: Training a nomrative model on different sites and make predictions on the same sites (equal distribution of sites in training and test data)

The full model contains a training set (covariates, responses, batch effects) and test set (covariates, responses, batch effects). All batch effects are distributes equally across training and test set.

Reqires:


In [ ]:
# train the model

ptk.normative.estimate(covfile=covfile, 
                       respfile=respfile,
                       tsbefile=tsbefile, 
                       model_type='bspline',
                       linear_sigma = 'True',
                       random_intercept_sigma = 'True',
                       random_slope_sigma = 'True',
                       linear_mu='True',
                       random_intercept_mu='True',
                       random_slope_mu='True',
                       alg='hbr', 
                       log_path=log_dir, 
                       binary=True,
                       output_path=output_path, 
                       testcov= testcovfile_path,
                       testresp = testrespfile_path,
                       trbefile=trbefile,
                       outputsuffix=outputsuffix, 
                       savemodel=True)

# Case 1: Results

In [ ]:
# load the estimated model for the first IDP (column)
model1_estimate = ldpkl(os.path.join(processing_dir, "Models/NM_0_0_estimate.pkl"))

In [ ]:
# we can always have a display of all available methods using the help call:
help(model1_estimate)

## Case 1a: plot the results for one site.

In [ ]:
# create dummy zscores
zscores = np.arange(-3,4)[:,np.newaxis]

# We need to provide a matrix of shape [N,d]. In this case d=1, so we need to expand this matrix with a single 'empty' dimension.
# We can do this by using the np.newaxis in this way.
# Create sythetic X achsis
n_synthetic_samples = 200
synthetic_X = np.linspace(0.15, 0.85, n_synthetic_samples)[:,np.newaxis]
# choose type of site effect by cerating intercept offset (or not)
be = np.zeros((n_synthetic_samples,2))
sevens = np.ones((n_synthetic_samples,1))*8
be[:,0] = sevens[:,0]
be=be.astype(int)

In [ ]:
# get quantiles based on x achsis, batch effetcs be and zscore range
q = model1_estimate.get_mcmc_quantiles(synthetic_X, be, zscores)

In [ ]:
# Plot the original training set data points (site 0) into the centiles
plt.scatter(X_test[batch_effects_test[:,0]==8], Y_test[batch_effects_test[:,0]==8, 0])
for i, v in enumerate(zscores):
    thickness = 1
    linestyle = "-"
    if v == 0:
        thickness = 2
    if abs(v) > 2:
        linestyle = "--"
    plt.plot(synthetic_X, q[i], linewidth = thickness, linestyle = linestyle, color = 'black', alpha = 0.7)

plt.title('site ' + str(be[0]))

## Case 1b: Add data points from site 7 into the quantile plot of site 8.

1. Create a forward prediction for the covariate combination of each individual of site 7 for site 8
2. Use yhat and s2 to invert z score of those individuals

Rationale:

The zscores that the model produces are normalized for site. Hence, the z-socres are comparable in z-score space.
In order to convert the data points back into different site spaces, we need to revert the z-score transformation.

$z_7 = z_8$





In [ ]:
# Select individuals from site 7

X_test_7=X_test[batch_effects_test[:,0]==7]
n_synthetic_samples = sum(batch_effects_test[:,0]==7)

# Create a batch effect file for forward predictions in site
be = np.zeros((n_synthetic_samples,2))
eights = np.ones((n_synthetic_samples,1))*8
be[:,0] = eights[:,0]
be=be.astype(int)

with open('X_test_7.pkl', 'wb') as file:
    pickle.dump(pd.DataFrame(X_test_7), file)

X_test_7_path = os.path.join(processing_dir, 'X_test_7.pkl')

with open('be_8.pkl', 'wb') as file:
    pickle.dump(pd.DataFrame(be), file)

be_8_path = os.path.join(processing_dir, 'be_8.pkl')

In [ ]:
sum(batch_effects_test[:,0]==7)

In [ ]:
ptk.normative.predict(covfile=X_test_7_path,
                      inputsuffix='_estimate',
                      respfile=None,
                      tsbefile=be_8_path,
                      model_path=output_path,
                      alg='hbr',
                      outputsuffix='_X_test_7')

### load the yhat and s2 for subjects from site 7 for site 8.

In [ ]:
yhat = ldpkl(os.path.join(processing_dir, "yhat_Xtest7.pkl"))
ys2 = ldpkl(os.path.join(processing_dir, "ys2_Xtest7.pkl"))

In [ ]:
# Plot the original test set data points (site 1) into the percentiles

plt.scatter(X_test[150:214], Y_test[150:214, 1])
for i, v in enumerate(zscores):
    thickness = 1
    linestyle = "-"
    if v == 0:
        thickness = 2
    if abs(v) > 2:
        linestyle = "--"
    plt.plot(synthetic_X, q[i], linewidth = thickness, linestyle = linestyle, color = 'black', alpha = 0.7)

plt.title('site ' + str(be[0]))

In [ ]:
# similar to above, but for the second site (np.ones)
zscores = np.arange(-4,4)[:,np.newaxis]

# We need to provide a matrix of shape [N,d]. In this case d=1, so we need to expand this matrix with a single 'empty' dimension.
# We can do this by using the np.newaxis in this way.
n_synthetic_samples = 200
synthetic_X = np.linspace(0.15, 0.85, n_synthetic_samples)[:,np.newaxis]
be = np.ones((n_synthetic_samples,1))*2
be

In [ ]:
# Load scoers from the model

yhat = ldpkl(os.path.join(processing_dir, "yhat_estimate.pkl"))
z_scores = ldpkl(os.path.join(processing_dir, "Z_estimate.pkl"))

In [ ]:
X_test.shape

# Transfer model

In [ ]:
outputsuffix= "_fit"
ptk.normative.fit(covfile=covfile, 
                       respfile=respfile,
                       tsbefile=tsbefile, 
                       model_type='bspline',
                       linear_sigma = 'True',
                       random_intercept_sigma = 'True',
                       random_slope_sigma = 'True',
                       linear_mu='True',
                       random_intercept_mu='True',
                       random_slope_mu='True',
                       alg='hbr', 
                       log_path=log_dir, 
                       binary=True,
                       output_path=output_path, 
                       #testcov=testcovfile_path,
                       #trbefile=trbefile,
                       #trbefile=be_path,
                       #testcov="covariate_forwardmodel.txt",
                       #testcov=synthetic_path,
                       #testresp = testrespfile_path,
                       outputsuffix=outputsuffix, 
                       savemodel=True)

In [ ]:
yhat_forward = ldpkl(os.path.join(processing_dir, "yhat_forward.pkl"))
yhat_forward

# Case 2: Transfer to new sites

The second case we are going to look at in this tutorial is to use a trained model to make a transfer to new, unseen sites. 
The 



In [ ]:
ptk.normative.transfer(covfile=testcovfile_path,
                      #covfile=synthetic_path,
                      inputsuffix='_estimate',
                      respfile=None,
                      tsbefile=tsbefile,
                      trbefile=trbefile,
                      model_path=output_path,
                      alg='hbr',
                      outputsuffix='_transfer')